### 방식

- 이미지가 존재하고, 그 안에 글자가 이미 labeling 되어 잇음.
- 입력, 출력(결과, 레이블) 존재 => 지도학습
- 데이터를 학습 -> 새로운 데이터 넣어서 예측
- **아직 접하지 않은 새로운 이미지 데이터를 넣어서 이것이 무엇인지를 예측**

### MNIST

- 손글씨 숫자 데이터 획득
- 학습용이 6만개 제공, 테스트 1만개 
- http://yann.lecun.com/exdb/mnist
> train-images-idx3-ubyte.gz:  training set images (9912422 bytes)   
> train-labels-idx1-ubyte.gz:  training set labels (28881 bytes)   
> t10k-images-idx3-ubyte.gz:   test set images (1648877 bytes)   
> t10k-labels-idx1-ubyte.gz:   test set labels (4542 bytes)  

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib import request as req
import os

# 1. 데이터 수집

In [2]:
# bs4를 이용해서 다운받을 데이터 link 4개를 수집한다.
url = 'http://yann.lecun.com/exdb/mnist/'
page = urlopen(url)
soup = BeautifulSoup(page, 'html5lib')
soup

<html><head>
   <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
   <title>MNIST handwritten digit database, Yann LeCun, Corinna Cortes and Chris Burges</title>
<script type="text/javascript">
var gaJsHost = (("https:" == document.location.protocol) ? "https://ssl." : "http://www.");
document.write(unescape("%3Cscript src='" + gaJsHost + "google-analytics.com/ga.js' type='text/javascript'%3E%3C/script%3E"));
</script><script type="text/javascript">
var pageTracker = _gat._getTracker("UA-6178702-1");
pageTracker._trackPageview();
</script></head>




<body alink="#111111" background="wm5b.gif" bgcolor="#FFFFFF" link="#FF0000" vlink="#820594">

<center>
<h1>
<font color="#FF0000"><font size="+4">THE MNIST DATABASE</font></font></h1></center>

<center><font color="#FF0000"><font size="+4">of handwritten digits</font></font></center>

<center><font color="#FF0000"><font size="+4"></font></font></center>

<center><font color="#3366FF"><font size="+1"><a href="http://yann

In [3]:
# a_tags=soup.select('p tt')

In [4]:
# [url+tag.a.string for tag in a_tags]

In [5]:
a_tags=soup.findAll('tt')[:4]

In [33]:
a_tags=[line.string for line in soup.select('p tt a')]

In [34]:
a_tags

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

In [36]:
downFiles = a_tags
downFiles

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

### 1-2 링크값을 이용하여 데이터를 직접 다운 받는다.

- 장점 : 언제 끝나는지, 얼마나 걸리는지 알 수 있다.
- 대용량 데이터를 받을때 유용

In [37]:
import gzip, os, os.path
#저장경로
savePath = './data/mnist/'

In [38]:
# 저장될 폴더가 없으면 만드시오.
if not os.path.exists(savePath):
    os.mkdir(savePath) #?? 까먹었으니 다시 해보자

In [10]:
# 웹상의 링크된 데이터를 다운받기 처리
from tqdm import tqdm_notebook
for fname in tqdm_notebook(downFiles):
    urlPath = url + fname
    locPath = savePath + fname
    print(urlPath,locPath)
    # 존재하면 받지 않겠다 -> 서비스시에는 이어받기, 지우기 받기 등등 필요
    if not os.path.exists(locPath):
        req.urlretrieve(urlPath,locPath)  # retrieve검색하다, 되찾아오다. 

http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz ./data/mnist/train-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz ./data/mnist/train-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz ./data/mnist/t10k-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz ./data/mnist/t10k-labels-idx1-ubyte.gz



In [11]:
# 압축해제
# gzip 파일 압축 해제한다.-> 누구를, 어디다가 풀건데?
for fname in tqdm_notebook(downFiles):
    #저장 파일(압축 해제 할 대상) 
    gzip_path = savePath + fname
    #압축해제 폴더
    rawPath = gzip_path[:-1*len('.gz')]
#     if not os.path.exists(rawPath):# 이거 만들었는데 왜뜸?
#         os.mkdir(rawPath)    
    #압축 해제->gz을 읽어서 대상파일을 열어서 쓴다.
    # rb : 바이너리 읽기 모드로 파일 오픈
    with gzip.open(gzip_path,'rb') as fp:#?? gz 파일 자체가 binary 인가?
        # 내용 읽기(압축을 해제해서 읽었다.)
        tmp = fp.read() # 더블클릭
        # 풀기
        with open(rawPath, 'wb') as fw:
            fw.write(tmp)    

TRAINING SET LABEL FILE (train-labels-idx1-ubyte):  
[offset] [type]          [value]          [description]   
0000     32 bit integer  0x00000801(2049) magic number (MSB first)  ->표식
0004     32 bit integer  60000            number of items   
0008     unsigned byte   ??               label   
0009     unsigned byte   ??               label  
........   
xxxx     unsigned byte   ??               label  
The labels values are 0 to 9.  
(32bit)4byte  
(32bit)4byte -> 60000개  
파일크기 60008 byte (60000)개 => 개당 1byte 

unsigned-> 양수  
signed -> 음수부터 양수까지  

TRAINING SET IMAGE FILE (train-images-idx3-ubyte):  
[offset] [type]          [value]          [description]   
0000     32 bit integer  0x00000803(2051) magic number   
0004     32 bit integer  60000            number of images=> 갯수   
0008     32 bit integer  28               number of rows =>가로 픽셀수  
0012     32 bit integer  28               number of columns  =>세로 픽셀수  
0016     unsigned byte   ??               pixel  
0017     unsigned byte   ??               pixel  
........ 
xxxx     unsigned byte   ??               pixel  

28 * 28 * 60000 = 4704000  
압축해제한 훈련데이터 파일의 크기는  
47040000 + 4*4 = 47040016  바이트)  


In [12]:
# 압축 해제된 데이터를 우리가 분석할 수 있는 데이터로 만들어준다.
# 파일 읽기 자체는 'rb' 
# 구문해석(헤더) 부분은 struct를 이용해서 간편하게 해석 
import struct
# 테스트 라벨 : t10k-labels-idx1-ubyte
path = savePath + downFiles[0][:-3]
path
fp = open(path, 'rb')
LABEL_HEADER_SIZE = 8
header = fp.read(LABEL_HEADER_SIZE)
fp.close()

In [13]:
print(header)
# 문자열
# b''(binary), '', r''(정규식)


b'\x00\x00\x08\x03\x00\x00\xea`'


In [14]:
# 원하는 바이너리 수 만큼 읽어서 정수값으로 변환해라
# I : 1번(4바이트)
# II : 4바이트 읽고, 4바이트 읽고
## ordering(오더링), 에디안(endian) 컴퓨터가 메모리에 데이터를 연속으로 쓰는 방법
# 어느 쪽을 취할지는 cpu마다 다르고 os 마다 다르고, 네트워크 통신상(TCP/IP) 에도 다르다.
# > : 빅 에디안 : 00 00 EA 60
# < : 리틀 에디안 : 60 EA 00 00
# 빅에디안(데이터가 빅 에디안 방식이니까)방식으로 4바이트씩 2번 읽어서 정수로 2개 리턴해라.
magic_number, label_count = struct.unpack('>II',header)
magic_number, label_count

(2051, 60000)

<img src='./data/binary.PNG ' width=100%/>

In [15]:

path = savePath + downFiles[1][:-3]
path
fp = open(path, 'rb')
LABEL_HEADER_SIZE = 16
header = fp.read(LABEL_HEADER_SIZE)
fp.close()

In [16]:
mn, count, row_size, col_size = struct.unpack('>IIII',header)
mn, count, row_size, col_size

(2049, 60000, 83887105, 151126275)

In [17]:
def search_name(text):
    text

In [18]:
# typeName : train, t10k
# limit : 데이터를 읽어들일 총 수: 100 or 10000 .. 60000(헤더에서 알아서 계산)

def mnist_to_csv(typeName, limit):
    # 이미지 데이터 헤드 정보 4개
    data_file = None
    label_file=None
    for file in downFiles[:-3]:
        if file.find(typeName):
            if file.find('images'):
                data_file=file
            elif file.find('label'):
                label_file=file
    print()
    path = savePath + data_file
    fp = open(path, 'rb')
    LABEL_HEADER_SIZE = 16
    image_head=fp.read(LABEL_HEADER_SIZE)
    fp.close()  
    mn, count, row_size, col_size = struct.unpack('>IIII',image_head)
    # 레이블 데이터 헤드 정보 2개 추출 실습
    path = savePath + test_file
    fp = open(path, 'rb')
    LABEL_HEADER_SIZE = 8
    label_head=fp.read(LABEL_HEADER_SIZE)
    fp.close()       
    magic_number, label_count = struct.unpack('>II',label_head)
     
    

In [19]:
mnist_to_csv('train',10000)# 이미지 10000개, 레이블 10000개

TypeError: must be str, not NoneType

In [20]:
def mnist_to_csv(typeName, limit):
    # 이미지 데이터 헤더 정보 4개
    # 레이블 데이터 헤더 정보 2개 추출 실습    
       
    # 2. 파일 오픈
    imageData_fp = open('./data/mnist/%s-images-idx3-ubyte' % typeName, 'rb')
    labelData_fp = open('./data/mnist/%s-labels-idx1-ubyte' % typeName, 'rb')
    #레이블 파일이 저장되어 있는 csv(텍스트) 생성
    csv_fp = open('./data/%s_mnist.csv'%typeName,'w',encoding='utf8')
    # 3. 파일 읽기   
    import struct
    # mn을 같이 받은 이유는 미사용, 원래는 해당 포맷을 확인할때 사용하는 용도
    m, data_count, row_size, col_size = struct.unpack('>IIII',imageData_fp.read(4+4+4+4))
    n, label_count = struct.unpack('>II',labelData_fp.read(4+4))    
    # 4. 읽은 데이터에서 헤더 정보 추출
    # 28 * 28 
    ONE_IMAGE_PIXELS_SIZE =row_size * col_size
    for idx in range(limit):
        
        # 데이터수와 레이블 수가 동일 하다 전제        
        #반복수가 데이터총 수를 넘으면 break
        # ex) limit = 100, data_count : 1000 => 데이터는 총 100개(limit 반영)
        # ex) limit = 1000, data_count : 100 => 데이터는 총 100개(data_count 반영됨)
        if idx >= data_count:break
        #1. 레이블 읽기(레이블 데이터의 크기는 1byte로 확정), 값은 이사이에 존재 0-9
        #unsigned byte => B
        label_tmp, = struct.unpack('B', labelData_fp.read(1)) #unpack이 tuple을 리턴함.
        print(label_tmp)
        # 레이블 데이터를 csv에 담겠다. 구분자 , 
        # row : 레이블, 픽셀데이터, .........(28 x 28) 줄바꿈
        csv_fp.write(str(label_tmp)+',')
        # 픽셀 데이터 읽기=> 얼마나 읽어야 하는가? => 28 * 28 = 784
         
        
    # 5. 파일 닫는다.
    imageData_fp.close()
    labelData_fp.close()    
    csv_fp.close()
    return m, data_count, row_size, col_size, n, label_count

In [21]:
mnist_to_csv('train',10)# 이미지 10000개, 레이블 10000개

5
0
4
1
9
2
1
3
1
4


(2051, 60000, 28, 28, 2049, 60000)

In [24]:
def mnist_to_csv(typeName, limit): 
    imageData_fp = open('./data/mnist/%s-images-idx3-ubyte' % typeName, 'rb')
    labelData_fp = open('./data/mnist/%s-labels-idx1-ubyte' % typeName, 'rb')   
    csv_fp = open('./data/%s_mnist.csv'%typeName,'w',encoding='utf8') 
    import struct  
    m, data_count, row_size, col_size = struct.unpack('>IIII',imageData_fp.read(4+4+4+4))
    n, label_count = struct.unpack('>II',labelData_fp.read(4+4))    

    ONE_IMAGE_PIXELS_SIZE =row_size * col_size
    for idx in range(limit):
        if idx >= data_count:break    
        label_tmp, = struct.unpack('B', labelData_fp.read(1))        
        # label csv에 기록
        csv_fp.write(str(label_tmp)+',')        
        bImgData = imageData_fp.read(ONE_IMAGE_PIXELS_SIZE)
        # 읽어온 이미지 데이터한개 한개를 문자열로 변환 -> 리스트에 담는다.
        #map('함수','시퀀스타입') # result sequence
        listImgData = [map(lambda x:str(x),bImgData)]
        listImgData = [str(byte) for byte in bImgData]
        csv_fp.write(','.join(listImgData)+'\r\n')
        print(len(listImgData),listImgData[:5])
        #print(type(bImgData))#type - bytes
        # 이 데이터가 진짜 이미지 인지 파일로 떨어트려서 확인
        # pgm 헤더
        s = "P2 28 28 255\n"
        '''
        s += ' '.join(listImgData)
        pgm_name = './data/mnist/{0}-{1}-{2}.pgm'.format(typeName,idx,label_tmp)
        with open(pgm_name,'w',encoding='utf8') as f:
            f.write(s)
        '''    
    imageData_fp.close()
    labelData_fp.close()    
    csv_fp.close()
    return m, data_count, row_size, col_size, n, label_count

In [25]:
mnist_to_csv('t10k',10)# 이미지 10000개, 레이블 10000개

784 ['0', '0', '0', '0', '0']
784 ['0', '0', '0', '0', '0']
784 ['0', '0', '0', '0', '0']
784 ['0', '0', '0', '0', '0']
784 ['0', '0', '0', '0', '0']
784 ['0', '0', '0', '0', '0']
784 ['0', '0', '0', '0', '0']
784 ['0', '0', '0', '0', '0']
784 ['0', '0', '0', '0', '0']
784 ['0', '0', '0', '0', '0']


(2051, 10000, 28, 28, 2049, 10000)